In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import warnings

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [4]:
data_dir = Path('../단국대(사용중)')
feature_dir = Path('../단국대(사용중)')
sub_dir = Path('../단국대(사용중)')
val_dir = Path('../단국대(사용중)')
tst_dir = Path('../단국대(사용중)')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
n_fold = 5
n_class = 3
seed = 42

In [5]:
algo_name = 'lgbcv'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [6]:
df = pd.read_csv(feature_file, index_col=0)
print(df.shape)
df.head()

(400000, 24)


,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,airmass_u,class,u_r,u_g,g_r,g_z,g_i,d_dered_u,d_dered_g,d_dered_r,d_dered_i,d_dered_z,d_dered_ig,d_dered_zg,d_dered_rz,d_dered_iz,d_obs_det
id,,,,,,,,,,,,,,,,,,,,,,,,
0,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,2.9444,1.1898,0.0,4.2545,2.9272,1.3273,3.3972,2.6643,-0.1397,-0.0790,-0.0544,-0.0403,-0.0307,-2.6257,-3.3488,2.0462,0.7232,0
1,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,0.6931,1.2533,1.0,1.5997,0.9902,0.6096,0.8930,0.7935,-0.0857,-0.0574,-0.0410,-0.0322,-0.0343,-0.7683,-0.8698,0.2767,0.1016,0
2,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,1.0986,1.0225,0.0,1.2501,0.9610,0.2891,0.4754,0.4319,-0.1787,-0.1388,-0.0963,-0.0718,-0.0540,-0.3649,-0.3905,0.1440,0.0257,0
3,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,1.6094,1.2054,0.0,5.4395,4.4719,0.9676,1.5541,1.2938,-0.3070,-0.1941,-0.1339,-0.1003,-0.0795,-1.2000,-1.4395,0.5321,0.2395,1
4,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,2.6391,1.1939,0.0,5.4111,3.7543,1.6568,2.7165,2.3750,-0.6820,-0.2653,-0.1794,-0.1339,-0.1067,-2.2436,-2.5579,0.9871,0.3144,1


In [7]:
y = df[target_col].values[:320000]
df.drop(target_col, axis=1, inplace=True)
trn = df.iloc[:320000].values
tst = df.iloc[320000:].values
feature_name = df.columns.tolist()
print(y.shape, trn.shape, tst.shape)

(320000,) (320000, 23) (80000, 23)


In [8]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [9]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=1000,
                             num_leaves=64,
                             learning_rate=0.1, #원래는 0.1
                             min_child_samples=10,
                             subsample=.5,
                             subsample_freq=1,
                             colsample_bytree=.8,
                             random_state=seed,
                             n_jobs=-1)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 0.85366
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 0.74528
[3]	valid_0's multi_logloss: 0.662244
[4]	valid_0's multi_logloss: 0.598821
[5]	valid_0's multi_logloss: 0.54079
[6]	valid_0's multi_logloss: 0.491122
[7]	valid_0's multi_logloss: 0.448994
[8]	valid_0's multi_logloss: 0.412824
[9]	valid_0's multi_logloss: 0.382106
[10]	valid_0's multi_logloss: 0.355408
[11]	valid_0's multi_logloss: 0.332751
[12]	valid_0's multi_logloss: 0.316551
[13]	valid_0's multi_logloss: 0.30102
[14]	valid_0's multi_logloss: 0.286451
[15]	valid_0's multi_logloss: 0.272674
[16]	valid_0's multi_logloss: 0.260433
[17]	valid_0's multi_logloss: 0.25132
[18]	valid_0's multi_logloss: 0.241736
[19]	valid_0's multi_logloss: 0.235186
[20]	valid_0's multi_logloss: 0.228712
[21]	valid_0's multi_logloss: 0.222299
[22]	valid_0's multi_logloss: 0.216893
[23]	valid_0's multi_logloss: 0.212219
[24]	valid_0's multi_logloss: 0

[123]	valid_0's multi_logloss: 0.160148
[124]	valid_0's multi_logloss: 0.160193
[125]	valid_0's multi_logloss: 0.160142
[126]	valid_0's multi_logloss: 0.160148
[127]	valid_0's multi_logloss: 0.160153
[128]	valid_0's multi_logloss: 0.160161
[129]	valid_0's multi_logloss: 0.160135
[130]	valid_0's multi_logloss: 0.16018
[131]	valid_0's multi_logloss: 0.160087
[132]	valid_0's multi_logloss: 0.160056
[133]	valid_0's multi_logloss: 0.160059
[134]	valid_0's multi_logloss: 0.160092
[135]	valid_0's multi_logloss: 0.160088
[136]	valid_0's multi_logloss: 0.160145
[137]	valid_0's multi_logloss: 0.160084
[138]	valid_0's multi_logloss: 0.160083
[139]	valid_0's multi_logloss: 0.16005
[140]	valid_0's multi_logloss: 0.160008
[141]	valid_0's multi_logloss: 0.15999
[142]	valid_0's multi_logloss: 0.159953
[143]	valid_0's multi_logloss: 0.159935
[144]	valid_0's multi_logloss: 0.159958
[145]	valid_0's multi_logloss: 0.159992
[146]	valid_0's multi_logloss: 0.160022
[147]	valid_0's multi_logloss: 0.160015
[14

[37]	valid_0's multi_logloss: 0.176335
[38]	valid_0's multi_logloss: 0.175229
[39]	valid_0's multi_logloss: 0.174211
[40]	valid_0's multi_logloss: 0.173153
[41]	valid_0's multi_logloss: 0.171971
[42]	valid_0's multi_logloss: 0.171201
[43]	valid_0's multi_logloss: 0.170281
[44]	valid_0's multi_logloss: 0.169594
[45]	valid_0's multi_logloss: 0.168875
[46]	valid_0's multi_logloss: 0.168392
[47]	valid_0's multi_logloss: 0.167714
[48]	valid_0's multi_logloss: 0.167214
[49]	valid_0's multi_logloss: 0.166786
[50]	valid_0's multi_logloss: 0.166222
[51]	valid_0's multi_logloss: 0.165609
[52]	valid_0's multi_logloss: 0.165093
[53]	valid_0's multi_logloss: 0.16468
[54]	valid_0's multi_logloss: 0.164379
[55]	valid_0's multi_logloss: 0.163959
[56]	valid_0's multi_logloss: 0.163703
[57]	valid_0's multi_logloss: 0.163546
[58]	valid_0's multi_logloss: 0.163212
[59]	valid_0's multi_logloss: 0.163094
[60]	valid_0's multi_logloss: 0.162878
[61]	valid_0's multi_logloss: 0.162604
[62]	valid_0's multi_loglo

In [10]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

93.2981%


In [11]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(80000, 1)


,class
id,
320000,0
320001,0
320002,0
320003,0
320004,0


In [12]:
sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [13]:
sub[target_col].value_counts()

2    40947
0    29979
1     9074
Name: class, dtype: int64

In [14]:
sub.to_csv(sub_file)